In [3]:
import os
os.environ["NAPARI_ASYNC"] = "1"
#os.environ["NAPARI_PERFMON"] = "/Users/fukai/projects/napari-travali2/examples/prefmon.json"

In [4]:
import pandas as pd
import tensorstore as ts
import json
from copy import deepcopy
spec = {
    "driver": "zarr3",
    "kvstore": {
            "driver": "file",
            "path": "/Volumes/Extreme SSD/aligned_image5_labels.zarr"
    },
    'context': {
            'cache_pool': {
                'total_bytes_limit': 100_000_000
            }
        },
        'recheck_cached_data':False,
    }
labels = ts.open(spec).result()[:,0,:,:]
image_spec = deepcopy(spec)
image_spec['kvstore']['path'] = '/Volumes/Extreme SSD/aligned_image5_image.zarr'
image = ts.open(image_spec).result()[:,0,:,:]
bboxes_df = pd.read_csv('/Volumes/Extreme SSD/aligned_image5_bboxes_df.csv')
display(bboxes_df.head())
with open('/Volumes/Extreme SSD/aligned_image5_labels.splits.json') as f:
    splits = json.load(f, object_hook=lambda d: {int(k): [int(_v) for _v in v] for k, v in d.items()})

,frame,label,min_y,min_x,max_y,max_x
0,0,1,13,1865,57,1909
1,0,2,19,4954,60,4994
2,0,3,29,2832,70,2876
3,0,4,35,1633,67,1669
4,0,5,38,2756,80,2797


In [5]:
import numpy as np
import tensorstore as ts
import trackarray_tensorstore as tats

print(labels.shape)
ta = tats.TrackArray(labels, splits, {}, bboxes_df)

(1965, 19991, 15437)


In [6]:
from napari import Viewer
from napari_travali2._stateful_widget import StateMachineWidget
from napari_travali2._logging import logger
import numpy as np

logger.setLevel("DEBUG")
logger.info("Starting napari-travali2")

viewer = Viewer()

In [7]:
subimage = image[:,4000:4000+2048, 4000:4000+2048]
sublabels = labels[:,4000:4000+2048, 4000:4000+2048]
viewer.add_image(subimage, name='image')
viewer.add_labels(sublabels, name='labels')


In [11]:
layer = viewer.layers["labels"]
layer._label_colors

AttributeError: 'Labels' object has no attribute '_label_colors'

In [39]:
class TensorStoreWrapper:
    def __init__(self, ts):
        self.ts = ts
        self.shape = ts.shape
        self.dtype = np.dtype(str(ts.dtype.name))
        self.chunks = ts.chunk_layout.read_chunk.shape
        
    def __getitem__(self, key):
        return self.ts[key]
    def __setitem__(self, key, value):
        self.ts[key] = value
    def __delitem__(self, key):
        del self.ts[key]
labels2 = TensorStoreWrapper(labels)
labels_da = da.from_array(labels2, chunks=labels2.chunks,
                          meta = np.array(labels2[:10,:10,:10]))
assert (labels_da[0].compute() == np.array(labels[0])).all()

In [40]:
sublabels_da = labels_da[:,4000:4000+2048, 4000:4000+2048]
sublabels_mapped = create_mapped_array(sublabels_da, {129893:129893})

In [ ]:
widget = StateMachineWidget(viewer, ta, image, crop_size=2048)
viewer.window.add_dock_widget(widget, area="right")
viewer.dims.set_current_step(0,0)

/Users/fukai/projects/napari-travali2/.venv/lib/python3.13/site-packages/napari/_vispy/layers/scalar_field.py:198: UserWarning: data shape (19991, 15437) exceeds GL_MAX_TEXTURE_SIZE 16384 in at least one axis and will be downsampled. Rendering is currently in 2D mode.
  warnings.warn(
INFO:napari_travali2._logging:Region clicked
INFO:napari_travali2._logging:clicked at [    0 11429  4441]
INFO:napari_travali2._logging:Region selected: coords [    0 11429  4441]
INFO:napari_travali2._logging:window selected: (slice(None, None, None), slice(np.int64(10405), np.int64(12453), None), slice(np.int64(3417), np.int64(5465), None))
INFO:napari_travali2._logging:Track clicked
INFO:napari_travali2._logging:button released
DEBUG:napari_travali2._logging:world coordinates: (np.float64(0.0), np.float64(11352.997118652294), np.float64(4416.266295215537))
DEBUG:napari_travali2._logging:data coordinates: [  0.         947.99711865 999.26629522]
INFO:napari_travali2._logging:clicked at [  0 948 999] at 

INFO:napari_travali2._logging:Region clicked
INFO:napari_travali2._logging:clicked at [   0 8961 5237]
INFO:napari_travali2._logging:Region selected: coords [   0 8961 5237]
INFO:napari_travali2._logging:window selected: (slice(None, None, None), slice(np.int64(7937), np.int64(9985), None), slice(np.int64(4213), np.int64(6261), None))
INFO:napari_travali2._logging:Track clicked
INFO:napari_travali2._logging:button released
DEBUG:napari_travali2._logging:world coordinates: (np.float64(0.0), np.float64(8944.734322482238), np.float64(5216.96805923945))
DEBUG:napari_travali2._logging:data coordinates: [   0.         1007.73432248 1003.96805924]
INFO:napari_travali2._logging:clicked at [   0 1008 1004] at frame 0 and label value 872
INFO:napari_travali2._logging:Track selected: frame 0 value 872
INFO:napari_travali2._logging:Current daughter candidates count: 0
INFO:napari_travali2._logging:messagebox selected
INFO:napari_travali2._logging:select selected
INFO:napari_travali2._logging:Track